### Prepare Data for Model 

* Convert data to the format that the model can consume. This preparation process should be consistent for train, test and prediction dataset. 

#### Input
* train.parquet
* test.parquet
* encoder_num.json

#### Output
* train_X.parquet
* train_Y.parquet
* test_X.parquet
* test_Y.parquet

In [1]:
suppressMessages(library(dplyr))

# params
data_name = "train.parquet"
# data_name = "test.parquet"
type = "train"

In [2]:
# file and directory info
project_dir = dirname(getwd())
if (type == "train"){
    interim_folder = "/data/interim/"
    processed_folder = "/data/processed/"

    data_path = interim_folder
    encoder_path = processed_folder
} else if (type == "pred"){
    tmp_folder = "/tmp/"
    artifacts_folder = "/artifacts/"

    data_path = tmp_folder
    encoder_path = artifacts_folder
} else {
     stop('the value for args[2] is not supported.')
}    

# input
data = arrow::read_parquet(paste0(project_dir, data_path, data_name))
encoder_num = jsonlite::fromJSON(paste0(project_dir, encoder_path, "encoder_num.json")) %>% 
    jsonlite::fromJSON(.)

# functions 
PrepareData = function(data, encoder_num, var_y){
    # numeric variables
    var_num = names(encoder_num)
    # impute missing values with known value
    data_var_impute = purrr::map2(data[, var_num], encoder_num, function(.x, .y) ifelse(is.na(.x), .y$impute, .x)) %>% 
                                  as_tibble(.)
    # normalized data with known mean and std
    data_var_scale = purrr::map2(data_var_impute[, var_num], encoder_num, function(.x, .y) (.x - .y$mean)/.y$std ) %>% 
                                 as_tibble(.)
                                
    X = data_var_scale
                                 
    # response variable
    if (is.null(var_y)){
        return(list(X = X))
    } else {
        Y = data %>% select(!!!var_y)
        return(list(X = X, Y = Y))
    }
}
                                 
# output and save
var_y = "quality"
if (!var_y %in% names(data)){
    var_y = NULL
}
feature_data = PrepareData(data, encoder_num, var_y = var_y)
name = unlist(strsplit(data_name, "[.]"))[1]
  
arrow::write_parquet(feature_data$X, paste0(project_dir, data_path, name, "_X.parquet"))
arrow::write_parquet(feature_data$Y, paste0(project_dir, data_path, name, "_Y.parquet"))

In [3]:
lapply(feature_data, head)
lapply(feature_data, dim)

$X
# A tibble: 6 x 11
  `fixed acidity` `volatile acidi… `citric acid` `residual sugar` chlorides
            <dbl>            <dbl>         <dbl>            <dbl>     <dbl>
1           0.284          -0.0284         0.341           0.125   -0.00414
2           1.55           -1.48           2.06            0.0156  -0.0455 
3          -0.750           1.56          -1.38           -0.0209  -0.398  
4           1.20            0.640         -0.164           0.198    0.369  
5          -0.405           0.139         -0.316          -0.240   -0.335  
6           2.12            0.584          1.15           -0.825   -0.335  
# … with 6 more variables: `free sulfur dioxide` <dbl>, `total sulfur
#   dioxide` <dbl>, density <dbl>, pH <dbl>, sulphates <dbl>, alcohol <dbl>

$Y
# A tibble: 6 x 1
  quality
    <int>
1       5
2       5
3       5
4       5
5       5
6       4

$X
[1] 1279   11

$Y
[1] 1279    1